In [1]:
import pandas as pd

In [12]:
!cd SPMM && python "smiles2emb.py" --input_file "../data/2025-04-12T12-32_export.csv" --output_file "lb170_embedded.csv" --seed 42 --checkpoint "../checkpoint_SPMM.ckpt" --device "cpu"

seed: 42
Loading CSV data
Creating model
LOADING PRETRAINED MODEL..
load checkpoint from ../checkpoint_SPMM.ckpt
_IncompatibleKeys(missing_keys=[], unexpected_keys=['temp'])
Generating SMILES embeddings...
Processed 100/172 SMILES
Processed 172/172 SMILES
CSV with embeddings saved to lb170_embedded.csv


In [96]:
data = pd.read_csv('SPMM/lb170_embedded.csv')

In [97]:
data['emb'] = data['emb'].apply(lambda x: eval(x))
data = data[['emb', 'PDSC']]

In [100]:
data = data[data['PDSC'] != 0]

In [102]:
import os
import joblib
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [103]:
os.makedirs('models_ridge_170lb', exist_ok=True)

emb_df = pd.DataFrame(data['emb'].tolist())
emb_df.columns = [f'emb_{i}' for i in range(emb_df.shape[1])]
X = emb_df

y = data['PDSC']

kf = KFold(n_splits=10, shuffle=True, random_state=42)
oof_predictions_ridge = np.zeros(len(data))

for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_ridge = Ridge(random_state=42, alpha=5)
    model_ridge.fit(X_train, y_train)

    oof_predictions_ridge[test_index] = model_ridge.predict(X_test)

    model_path = f'models_ridge_170lb/ridge_model_fold_{fold}.joblib'
    joblib.dump(model_ridge, model_path)

In [104]:
mse = mean_squared_error(y, oof_predictions_ridge)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, oof_predictions_ridge)
r2 = r2_score(y, oof_predictions_ridge)

print(f'''
Ridge Metrics
- MSE: {mse:.4f}
- RMSE: {rmse:.4f}
- MAE: {mae:.4f}
- R²: {r2:.4f}
''')


Ridge Metrics
- MSE: 1165.3725
- RMSE: 34.1376
- MAE: 24.5369
- R²: 0.6540

